In [1]:
import networkx as nx
from qa.generator import QAGenerator
from qa.templates import TEMPLATES

In [2]:
graph = nx.read_graphml("municipalities_peaks_castles.graphml")

In [3]:
generator = QAGenerator()
qas = generator.generate_questions(graph, TEMPLATES, num_questions=1, add_distractors=10)
for qa in qas:
	print(qa)
	for fact in qa.context_facts:
		print(fact)

Generated 1 questions after 1 attempts.
QA(question='V kateri statistični regiji stoji grad stari grad Polhov Gradec?', answer='osrednjeslovenska statistična regija', num_facts=12)
<osrednjeslovenska statistična regija> <has_municipality> <Občina Škofljica>.
<osrednjeslovenska statistična regija> <has_municipality> <Občina Ivančna Gorica>.
<osrednjeslovenska statistična regija> <has_municipality> <Občina Borovnica>.
<osrednjeslovenska statistična regija> <has_municipality> <Občina Litija>.
<Občina Dobrova - Polhov Gradec> <has_peak> <Grmada, Polhograjsko hribovje>.
<osrednjeslovenska statistična regija> <has_municipality> <Občina Logatec>.
<osrednjeslovenska statistična regija> <has_municipality> <Občina Brezovica>.
<osrednjeslovenska statistična regija> <has_municipality> <Občina Velike Lašče>.
<Občina Dobrova - Polhov Gradec> <located_in> <osrednjeslovenska statistična regija>.
<stari grad Polhov Gradec> <located_in> <Občina Dobrova - Polhov Gradec>.
<osrednjeslovenska statistična re